In [1]:
from __future__ import division
import theano
import numpy as np
import theano.tensor as T
import matplotlib as plt
%matplotlib inline

/Users/NatchaS/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


<function function at 0x10a32baa0>
